In [17]:
import numpy as np              # linear algebra
import pandas as pd             # data processing, CSV file I/O (e.g. pd.read_csv)
                                
import matplotlib.pyplot as plt # data visualization
import seaborn as sns           # data visualization
                                
from sklearn.model_selection import train_test_split

import optuna
import lightgbm as lgb
from sklearn.metrics import roc_curve, auc

# Data Preparation

---

## Data Extraction

In [2]:
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv")
train = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/test.csv")

In [3]:
sample_submission.shape

In [4]:
train.shape

In [5]:
train.info()

In [6]:
test.shape

In [7]:
test.info()

## Data Concatenation

In [8]:
data = pd.concat([train, test], sort = False)
data.shape

In [9]:
data.info()

In [10]:
data.head()

## Null Check

In [11]:
null_cols = [col for col in data.iloc[: , : -1].columns if data[col].isnull().sum() != 0]
null_cols

In [12]:
del data

# Data Visualization

---

## Features Distribution

In [13]:
float_cols = [col for col in train.iloc[: , 1 : -1].columns if train[col].dtype == "float64"]
len(float_cols)

In [14]:
CHUNMEIHONG = '#f1939c'
QIUBOLAN = '#8abcd1'
XIANGYABAI = '#fffef8'
ZHENZHUHUI = '#e4dfd7'

fig, axes = plt.subplots(20, 5, figsize = (16, 48))
axes = axes.flatten()

def features_distribution(axes):
    for idx, ax in enumerate(axes):
        sns.kdeplot(
            data = train[float_cols + ['target']],
            ax = ax,
            hue = 'target',
            fill = True,
            x = f'f{idx}',
            palette = [f'{CHUNMEIHONG}', f'{QIUBOLAN}'],
            legend = idx == 0,
            alpha = .5,
            linewidth = 2.5,
        )
        
        ax.grid(
            color = XIANGYABAI,
            linestyle = ":",
            linewidth = 1.25,
            alpha = 0.3,
        )
        ax.set_facecolor(ZHENZHUHUI)
        #ax.set_xticks([])
        #ax.set_yticks([])
        ax.spines['left'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.yaxis.tick_right()
        ax.yaxis.set_label_position("left")
        ax.set_title(
            f'f{idx}',
            loc = 'right',
            weight = 'bold',
            fontsize = 10,
        )
        #ax.set_xticks([])
        #ax.set_yticks([])
        ax.set_xlabel('')
        #ax.set_ylabel('')
        if idx % 5 != 0:
            ax.set_ylabel('')

features_distribution(axes)

fig.supxlabel('Probability', ha = 'center', fontweight = 'bold', fontsize = 16, y = -0.005,)
fig.supylabel('Density', ha = 'center', fontweight = 'bold', fontsize = 16, x = -0.005,)
fig.suptitle('Features Distribution', ha = 'center', fontweight = 'heavy', fontsize = 20, y = 1,)
fig.tight_layout()

In [15]:
X = train.copy()
y = X.pop('target')
X_test = test.copy()

del train, test

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2,
                                                      random_state =0, stratify = y)

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference = lgb_train)

def objective(trial):
    params = {
        'metric': 'auc',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
        'max_bin': trial.suggest_int('max_bin', 128, 255),
        'num_leaves': trial.suggest_int('num_leaves', 32, 64),
        'device': 'gpu',
    }
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference = lgb_train)
    
    model = lgb.train(params,
                      lgb_train,
                      valid_sets = [lgb_train, lgb_eval],
                      verbose_eval = 10,
                      num_boost_round = 1000,
                      early_stopping_rounds = 10)
    y_pred_valid = model.predict(X_valid, num_iteration = model.best_iteration)
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_valid, y_pred_valid)
    score = auc(false_positive_rate, true_positive_rate)
    return score

study = optuna.create_study(sampler = optuna.samplers.RandomSampler(seed = 0))
study.optimize(objective, n_trials = 40)

In [ ]:
study.best_params

In [ ]:
params = {
    'metric': 'auc',
    'learning_rate': study.best_params['learning_rate'],
    'num_leaves': study.best_params['num_leaves'],
    'max_bin': study.best_params['max_bin'],
    'device': 'gpu',
}


lgb_train = lgb.Dataset(X_train, y_train,)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference = lgb_train,)

model = lgb.train(params,
                  lgb_train,
                  valid_sets = [lgb_train, lgb_eval],
                  verbose_eval = 10,
                  num_boost_round = 1000,
                  early_stopping_rounds = 10)


y_pred = model.predict(X_test, num_iteration = model.best_iteration)

In [16]:
sub = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv")
sub['target'] = y_pred
sub.to_csv('submission_1st_trial.csv', index = False)

In [ ]:
sub.head()

In [ ]:
float_cols